#Import der notwenigen Pakete

In [3]:
from pylab import *
import matplotlib.animation as animation
from matplotlib.widgets import Slider, Button, RadioButtons

#Vorbereitende Funktion

In [4]:
def prepGoL (size = [9,9], bouncon = 1, incon = [[3,3],[3,4],[3,5]], rule = 'B3/S23'):
    """Die Funktion prepGoL bereitet Textdateien vor, in welche die Bedingungen zum Ausführen von GameOfLife benötigt werden.
    Es sind standrtmäßige Werte eingesetzt, welche nach belieben verändert werden können. Die Veränderung können sowohl durch Veränderung der Parameter der Funktion oder durch Bearbeitung des erstellten Dokuments vorgenommen werden.  
    kwargs:
        size = [9,9] : Systemgröße
        bouncon = 1 : Randbedingungen
        incon = [[3,3],[3,4],[3,5]] : Anfangsbedingungen
        rule = 'B3/S23' : Regelstring
        
    Hinweise: Es ist eine Form nach folgendem Muster für die Anweisungen zu wählen:
        size: [Höhe, Breite]
        bouncon:
            0 - periodisch
            1 - begrenzt
            2 - unendlich (dynamisch)
        incon: [[xpos,ypos], [xpos,ypos], ...]
        rule: B3/S23
            B - Anzahlen von Nachbarzellen zur Geburt - /S - Anzahlen von Nachbarzellen zum Überleben vorhandener Zellen
    kein return Wert
    """
    #Größe
    a = open('GameOfLifeSize.dat','w')
    a.write('#Systemgröße als zwei natürliche Zahlen; Höhe und Breite (Höhe Breite)\n')
    for i in range(2):
        a.write('{} '.format(size[i]))
    a.write("\n")
    #Randbedingung
    b = open('GameOfLifeBouncon.dat','w')
    b.write('#Randbedingungen als Zahl:\n')
    b.write('#0 - periodisch\n')
    b.write('#1 - begrenzt\n')
    b.write('#2 - unendlich\n')
    b.write('{}\n'.format(bouncon))
    #Anfangsbedingung
    c = open('GameOfLifeIncon.dat','w')
    c.write('#Anfangsbedingungen; Positionen der einzelnen besetzten Zellen untereinander: xpos ypos\n')
    for i in range(len(incon)):
        for j in range(2):
            c.write('{} '.format(incon[i][j]))
        c.write("\n")
    #Regel, aber nur als Zahlen, nicht als String
    d = open('GameOfLifeRule.dat','w')
    d.write('#Regeln für das Leben der Zellen: z.B. "3 ENTER 2 3"\n')
    d.write('#erste Zeile: Anzahlen von Nachbarzellen zur Geburt \n#zweite Zeile: Anzahlen von Nachbarzellen zum Überleben vorhandener Zellen\n')
    d.write('#Anschließend sind die Zeilen so mit 0en aufzufüllen, dass sie gleich lang sind\n')
    d.write('#Sollte Null als Bestandteil dr Regel gewählt werden, so muss 0 als erster Wert dieser Zeile eingetragen werden!\n')
    for i in range(2):
        for j in range(max(len(rule.split("/")[1])-1, len(rule.split("/")[0])-1)):
            try:
                d.write('{} '.format(int(rule.split("/")[i-1][j+1])))
            except:
                d.write('0')
        d.write("\n")
    a.close()
    b.close()
    c.close()
    d.close()

#Durchführung eines Schrittes

In [5]:
def GoLStep(M, livecrit, borncrit, bouncon):
    """
    Die Funktion GoLStep führt einen einzelnen Schritt des Programms GameOfLife durch.
    args:
        M : Matrix, welche die Daten des Feldes entählt (0 für tote und 1 für lebende Zellen)
        livecrit : Liste mit Kriterien, wann eine lebende Zelle am Leben bleibt. (Liste mit int Zahlen)
        borncrit : Liste mit Kriterien, wann eine neue Zelle geboren wird. (Liste mit int Zahlen)
        bouncon : Randbedingungen (int Zahl)
                    0 - periodisch
                    1 - begrenzt
                    2 - unendlich (dynamisch) 
    
    returns newM (neu, nach den vorgegebenen Regeln berechnete Matrix / neues Feld)
            breite (Breite dieser Matrix)
            hohe (Höhe dieser Matrix)
    """
    hohe, breite=M.shape
    if bouncon==2: #unendliche Randbedingung
        l = M[:,0].dot(np.ones(hohe)) #links
        r = M[:,-1].dot(np.ones(hohe)) #rechts
        o = M[0].dot(np.ones(breite)) #oben
        u = M[-1].dot(np.ones(breite)) #unten
        #Ergänzung der Matrix an entsprechender Stelle
        if not l==0:
            M = np.c_[np.zeros(hohe), M]
            breite=breite+1
        if not r==0:
            M = np.c_[M, np.zeros(hohe)]
            breite=breite+1
        if not o==0:
            M = np.r_[np.zeros([1,breite]),M]
            hohe=hohe+1
        if not u==0:
            M = np.r_[M,np.zeros([1,breite])]
            hohe=hohe+1
        #Erstellung der Matrizen zur Berechnung der Anzahl der Nachbarzellen
        neighbor1 = np.eye(hohe)+np.eye(hohe, k=1)+np.eye(hohe, k=-1)
        neighbor2 = np.eye(breite)+np.eye(breite, k=1)+np.eye(breite, k=-1)
    elif bouncon==1: #endliche Randbedingung
        neighbor1 = np.eye(hohe)+np.eye(hohe, k=1)+np.eye(hohe, k=-1)
        neighbor2 = np.eye(breite)+np.eye(breite, k=1)+np.eye(breite, k=-1)
    elif bouncon==0: #periodische Randbedingung
        neighbor1 = np.eye(hohe)+np.eye(hohe, k=1)+np.eye(hohe, k=-1)
        neighbor1[0,-1]=1
        neighbor1[-1,0]=1
        neighbor2 = np.eye(breite)+np.eye(breite, k=1)+np.eye(breite, k=-1)
        neighbor2[0,-1]=1
        neighbor2[-1,0]=1
    neighbors = neighbor1.dot(M).dot(neighbor2)-M #Matrix mit Anzahl der Nachbarzellen jeder Zelle
    livenei = M*neighbors #Matrix, welche die Nachbarzahlen der lebenden Zellen enthält
    deadnei = neighbors*(1-M) #Matrix, welche die Nachbarzahlen der toten Zellen enthält
    newM = np.zeros((hohe,breite))
    for i in livecrit:
        newM[livenei==i]=1
    for i in borncrit:
        newM[deadnei==i]=1
    return newM, breite, hohe

#Animierter Plot mit Zusatzfunktionen

In [6]:
def GoLplot(field, step, lcrit, bcrit, bcon, interval, speed):
    """
    Die Funktion GoLplot stellt die Entwicklung des Feldes nach vorgegebener Regel dar.
    args:
        field : Ausganagsmatrix (Feld zum Schritt 0) mit den Daten der lebenden Zellen (1) und der toten Zellen (0)
        step : Funktion, welche die Matrix zum nächsten Zeitpunkt bestimmt (mit den args *[field, livecrit, borncrit, bouncon])
        lcrit : Liste mit Kriterien, wann eine lebende Zelle am Leben bleibt. (Liste mit int Zahlen)
        bcrit : Liste mit Kriterien, wann eine neue Zelle geboren wird. (Liste mit int Zahlen)
        bouncon : Randbedingungen (int Zahl)
                    0 - periodisch
                    1 - begrenzt
                    2 - unendlich (dynamisch)  
        interval : Anzahl der durchzuführenden Schritte (int Zahl)
        speed : Schnelligkeit, mit der die Animation gezeigt werden soll
                    float Zahl, welche die Verweildauer in ms angibt 
    
    kein return Wert
    """
    global borncrit
    borncrit=bcrit
    global livecrit
    livecrit=lcrit
    global fields
    fields=[np.array(field)]
    ihohe, ibreite=field.shape
    global hohen
    hohen=[ihohe]
    global breiten
    breiten=[ibreite]
    global pause
    global editing
    global single_step
    global single_b_step
    global bouncon
    bouncon=bcon
    #Plot
    fig, ax = subplots()
    ax.set_position([0.3,0.25,0.69,0.69])
    line = ax.imshow(field, interpolation='nearest')    
    #Slider für die Schrittnummer
    slideri = axes([0.35, 0.1, 0.6, 0.03])
    global reali
    reali = Slider(slideri, 'i', 0.0, interval, valinit=0)
    #Buttons zur Funktionswahl
    opt1 = axes([0.025, 0.85, 0.25, 0.1])
    GenOpt = Button(opt1, 'General Options')
    opt2 = axes([0.025, 0.7, 0.25, 0.1])
    ColOpt = Button(opt2, 'Colors')
    opt3 = axes([0.025, 0.55, 0.25, 0.1])
    FieldOpt = Button(opt3, 'Edit Field')
    opt4 = axes([0.025, 0.4, 0.25, 0.1])
    NewF = Button(opt4, 'New Field')
    startax = axes([0.025, 0.25, 0.1, 0.1])
    startb = Button(startax, '>')
    pauseax = axes([0.15, 0.25, 0.1, 0.1])
    pauseb = Button(pauseax, '||')
    singlestepax = axes([0.15, 0.1, 0.1, 0.1])
    singlestepb = Button(singlestepax, '>>')
    singlebstepax = axes([0.025, 0.1, 0.1, 0.1])
    singlebstepb = Button(singlebstepax, '<<')
    #Setzen der cmap auf schwarze lebende Zellen auf weißem Hintergrund
    col = {'red': [(0.0,  0.0, 0.0),
                   (1.0,  0, 0)],

         'green': [(0.0,  0.0, 0.0),
                   (1.0,  0, 0)],

         'blue':  [(0.0,  0.0, 0.0),
                   (1.0,  0, 0)],

         'alpha': [(0.0,  0.0, 0.0),
                   (1.0,  1.0, 1.0)] }
    cm = mpl.colors.LinearSegmentedColormap("cm", col)
    register_cmap(cmap=cm)
    set_cmap(cm)
    
    #Funktionen für die verschiedenen Optionen (werden in neuem Fenster geöffnet)
    #General Options
    def genopt(val):
        """
        Die Funktion genopt öffnet Buttons / Radiobuttons zum Ändern der Regel und der Randbedingung in einem neuen Fenster.
        args:
            val : Benötigt zum Ausführen mit Buttons
        kein return Wert
        """
        global bouncon
        #Randbedingung
        fig, bounconax = subplots()
        bounconax.set_position([0.1, 0.3, 0.35, 0.4])
        bounconax.set_title("Boundary Condition")
        bounconb = RadioButtons(bounconax, ('periodical', 'finite', 'infinite'), active=bouncon)
        #Regel mit je 9 Buttons
        for i in range(9):
            exec("borncritax{} = axes([0.5, 0.{}, 0.1, 0.1])".format(i,8-i))
            exec("global borncrit{}; borncrit{} = Button(borncritax{}, '{}')".format(i,i,i,i))
            exec("livecritax{} = axes([0.75, 0.{}, 0.1, 0.1])".format(i,8-i))
            exec("global livecrit{}; livecrit{} = Button(livecritax{}, '{}')".format(i,i,i,i))
            if i in borncrit:
                exec("borncrit{}.color=[1,1,0]".format(i))
            else:
                exec("borncrit{}.color=[1,1,1]".format(i))
            if i in livecrit:
                exec("livecrit{}.color=[1,1,0]".format(i))
            else:
                exec("livecrit{}.color=[1,1,1]".format(i))
            exec("""def borncritfunc{}(val):
                global borncrit
                global borncrit{}
                if {} in borncrit:
                    borncrit.remove({})
                    borncrit{}.color=[1,1,1]
                else:
                    borncrit.append({})
                    borncrit{}.color=[1,1,0]
                global fields
                global reali
                fields=[fields[int(reali.val)]]
                ihohe, ibreite=fields[0].shape
                global hohen
                hohen=[ihohe]
                global breiten
                breiten=[ibreite]
                reali.set_val(0)""".format(i,i,i,i,i,i,i))
            exec("borncrit{}.on_clicked(borncritfunc{})".format(i,i))
            exec("""def livecritfunc{}(val):
                global livecrit
                global livecrit{}
                if {} in livecrit:
                    livecrit.remove({})
                    livecrit{}.color=[1,1,1]
                else:
                    livecrit.append({})
                    livecrit{}.color=[1,1,0]
                global fields
                global reali
                fields=[fields[int(reali.val)]]
                ihohe, ibreite=fields[0].shape
                global hohen
                hohen=[ihohe]
                global breiten
                breiten=[ibreite]
                reali.set_val(0)""".format(i,i,i,i,i,i,i))
            exec("livecrit{}.on_clicked(livecritfunc{})".format(i,i))
            exec("borncritax0.set_title('born-criteria')")
            exec("livecritax0.set_title('live-criteria')")
        show()
    
        def bounconfunc(bcon):
            '''
            Die Funktion bounconfunc ändert die Animation entsprechend der neuen Randbedingung.
            Sie wird von vorne gestartet.
            args:
                bcon : gewählte Randbedingung
            kein return Wert
            '''
            global fields
            global reali
            fields=[fields[int(reali.val)]]
            ihohe, ibreite=fields[0].shape
            global hohen
            hohen=[ihohe]
            global breiten
            breiten=[ibreite]
            global bouncon
            if bcon in ['finite',1]:
                bouncon=1
            elif bcon in ['infinite',2]:
                bouncon=2
            elif bcon in ['periodical',0]:
                bouncon=0
            reali.set_val(0)
        bounconb.on_clicked(bounconfunc)
    
    def colors(val):
        """
        Die Funktion colors öffnet Slider / Radiobuttons zum Ändern cmap. Es wird immer ein weißer Hintergrund mit Zellen der entsprechenden Farbe eingestellt.
        args:
            val : Benötigt zum Ausführen mit Buttons
        kein return Wert
        """
        fig, sliderr = subplots()
        sliderr.set_position([0.35, 0.35, 0.6, 0.03])
        sliderg = axes([0.35, 0.498, 0.6, 0.03])
        sliderb = axes([0.35, 0.647, 0.6, 0.03])
        colors = axes([0.025,0.2,0.25,0.6])

        r = Slider(sliderr, 'red', 0.0, 1.0, valinit=0)
        g = Slider(sliderg, 'green', 0.0, 1.0, valinit=0)
        b = Slider(sliderb, 'blue', 0.0, 1.0, valinit=0)
        colorbuttons = RadioButtons(colors, ('black', 'green', 'red', 'blue', 'yellow'))
        show()
        
        def colorslide(val):
            '''
            Die Funktion colorslide verändert das Farbschema gemäß den auf den Slidern eingestellten Werten.
            args: 
                val : Nötig zum Ausführen mit Buttons
            kein return Wert
            '''
            red=r.val
            green=g.val
            blue=b.val
            col = {'red': [(0.0,  0.0, 0.0),
                           (1.0,  red, red)],

                 'green': [(0.0,  0.0, 0.0),
                           (1.0,  green, green)],

                 'blue':  [(0.0,  0.0, 0.0),
                           (1.0,  blue, blue)],

                 'alpha': [(0.0,  0.0, 0.0),
                           (1.0,  1.0, 1.0)] }
            cm = mpl.colors.LinearSegmentedColormap("cm", col)
            register_cmap(cmap=cm)
            set_cmap(cm)

        def colorbut(col='black'):
            '''
            Die Funktion colorbut stellt das Farbschema entsprechend der gewählten Farbe ein.
            kwargs:
                col='black' : gewählte Farbe
            kein return Wert
            '''
            #Einstellen der Slider
            if col=="red":
                r.set_val(1)
                g.set_val(0)
                b.set_val(0)
            elif col=="green":
                r.set_val(0)
                g.set_val(1)
                b.set_val(0)
            elif col=="blue":
                r.set_val(0)
                g.set_val(0)
                b.set_val(1)
            elif col=="black":
                r.set_val(0)
                g.set_val(0)
                b.set_val(0)
            elif col=="yellow":
                r.set_val(1)
                g.set_val(1)
                b.set_val(0)
            #Setzen des Farbschemas durch die Werte der Slider
            colorslide(0)
        
        r.on_changed(colorslide)
        g.on_changed(colorslide)
        b.on_changed(colorslide)
        colorbuttons.on_clicked(colorbut)
    
    edfia = axes([0.3875,0.26,0.51,0.685])
    edfib = Button(edfia, "0")
    edfia.set_visible(0)
    def edfi(val):
        """
        Die Funktion edfi passt den Button edfib in der Größe an, sodass er sich nahezu deckungsgleich mit dem Plot des Feldes befindet.
        editing wird auf True gesetzt, was das Editieren möglich macht.
        args:
            val : Nötig zum Ausführen mit Buttons
        kein return Wert
        """
        global editing
        editing=True
        global reali
        i=int(reali.val)
        global hohen
        global breiten
        hohe=hohen[i]
        breite=breiten[i]
        if hohe/breite>0.7: #Der Button wird der Größe angepasst
            edfia.set_position([0.3875+0.685*(1-breite/hohe)/2,0.254,0.51*breite/hohe,0.685])
        else:
            edfia.set_position([0.305,0.2+0.51*(1-hohe/breite)/2,0.68,0.51*(hohe/breite+0.5)])
    
    def edfunc(event):
        """
        Die Funktion edfunc erwirkt die Veränderung des Wertes an der gewählten Stelle.
        Lebende Zellen werde tot, tote Zellen lebendig.
        Sie funktioniert nur, wenn editing auf True gesetzt wurde. (siehe edfi)
        args:
            event : Event des Buttons, die Position des Klicks wird abgerufen.
        kein return Wert
        """
        if editing:
            global reali
            i=int(reali.val)
            global hohen
            global breiten
            hohe=hohen[i]
            breite=breiten[i]
            ydata=int(event.xdata*breite)
            xdata=int((1-event.ydata)*hohe)
            reali.set_val(0)
            global fields
            fields[i][xdata,ydata]=abs(fields[i][xdata,ydata]-1) #Änderung des Wertes der Matrix an der entsprechenden Stelle
            fields=[fields[i]]
            hohen=[hohen[i]]
            breiten=[breiten[i]]
    edfib.on_clicked(edfunc)
    
    def NewFfunc(val):
        """
        Die Funktion NewFfunc öffnet Buttons zum Auswählen der Größe des zu erstellenden Feldes.
        Nach Bestätigung des x-Wertes kann der y-Wert eingegeben werden. Nach nochmaliger Bestätigung wird ein leeres Feld der eingegebenen Größe erzeugt.
        args:
            val : Benötigt zum Ausführen mit Buttons
        kein return Wert
        """
        global newsizex
        newsizex=0
        global newsizey
        newsizey=0
        global xcom
        xcom=False
        #Ziffern von 0 bis 9 angeordnet wie in einem Nummernpad
        fig, num1a = subplots()
        num1a.set_position([0.25, 0.3, 0.1, 0.1])
        num1b = Button(num1a, '1')
        num2a = axes([0.45, 0.3, 0.1, 0.1])
        num2b = Button(num2a, '2')
        num3a = axes([0.65, 0.3, 0.1, 0.1])
        num3b = Button(num3a, '3')
        num4a = axes([0.25, 0.5, 0.1, 0.1])
        num4b = Button(num4a, '4')
        num5a = axes([0.45, 0.5, 0.1, 0.1])
        num5b = Button(num5a, '5')
        num6a = axes([0.65, 0.5, 0.1, 0.1])
        num6b = Button(num6a, '6')
        num7a = axes([0.25, 0.7, 0.1, 0.1])
        num7b = Button(num7a, '7')
        num8a = axes([0.45, 0.7, 0.1, 0.1])
        num8b = Button(num8a, '8')
        num9a = axes([0.65, 0.7, 0.1, 0.1])
        num9b = Button(num9a, '9')
        num0a = axes([0.45, 0.1, 0.1, 0.1])
        num0b = Button(num0a, '0')
        dela  = axes([0.25, 0.1, 0.1, 0.1]) #Button zum Löschen des letzten Wertes
        delb  = Button(dela, '<-')
        coma  = axes([0.65, 0.1, 0.1, 0.1]) #Enter-Button
        comb  = Button(coma, '$_{<-}|$')
        num7a.set_title('Please enter new size')
        num8a.set_title('xsize=')
        num9a.set_title('ysize=')
        show()
        #Funktionen zur Eingabe der Ziffern
        def num0func(val):
            global newsizex
            global newsizey
            global xcom
            if not xcom:
                newsizex = newsizex*10
                num8a.set_title(newsizex)
            else:
                newsizey = newsizey*10
                num9a.set_title(newsizey)
        def num1func(val):
            global newsizex
            global newsizey
            global xcom
            if not xcom:
                newsizex = newsizex*10+1
                num8a.set_title(newsizex)
            else:
                newsizey = newsizey*10+1
                num9a.set_title(newsizey)
        def num2func(val):
            global newsizex
            global newsizey
            global xcom
            if not xcom:
                newsizex = newsizex*10+2
                num8a.set_title(newsizex)
            else:
                newsizey = newsizey*10+2
                num9a.set_title(newsizey)
        def num3func(val):
            global newsizex
            global newsizey
            global xcom
            if not xcom:
                newsizex = newsizex*10+3
                num8a.set_title(newsizex)
            else:
                newsizey = newsizey*10+3
                num9a.set_title(newsizey)
        def num4func(val):
            global newsizex
            global newsizey
            global xcom
            if not xcom:
                newsizex = newsizex*10+4
                num8a.set_title(newsizex)
            else:
                newsizey = newsizey*10+4
                num9a.set_title(newsizey)
        def num5func(val):
            global newsizex
            global newsizey
            global xcom
            if not xcom:
                newsizex = newsizex*10+5
                num8a.set_title(newsizex)
            else:
                newsizey = newsizey*10+5
                num9a.set_title(newsizey)
        def num6func(val):
            global newsizex
            global newsizey
            global xcom
            if not xcom:
                newsizex = newsizex*10+6
                num8a.set_title(newsizex)
            else:
                newsizey = newsizey*10+6
                num9a.set_title(newsizey)
        def num7func(val):
            global newsizex
            global newsizey
            global xcom
            if not xcom:
                newsizex = newsizex*10+7
                num8a.set_title(newsizex)
            else:
                newsizey = newsizey*10+7
                num9a.set_title(newsizey)
        def num8func(val):
            global newsizex
            global newsizey
            global xcom
            if not xcom:
                newsizex = newsizex*10+8
                num8a.set_title(newsizex)
            else:
                newsizey = newsizey*10+8
                num9a.set_title(newsizey)
        def num9func(val):
            global newsizex
            global newsizey
            global xcom
            if not xcom:
                newsizex = newsizex*10+9
                num8a.set_title(newsizex)
            else:
                newsizey = newsizey*10+9
                num9a.set_title(newsizey)
        #Funktion zum Löschen der letzten Ziffer
        def delnum(val):
            global newsizex
            global newsizey
            global xcom
            if not xcom:
                newsizex=int(newsizex/10)
                num8a.set_title(newsizex)
            else:
                newsizey=int(newsizey/10)
                num9a.set_title(newsizey)
        #Funktion zum Bestätigen der Eingabe
        def comnum(val):
            global newsizex
            global newsizey
            global xcom
            if not xcom: #Bestätigung des x-Wertes und Übergang zur Eingabe des y-Wertes
                num8a.set_title(str(newsizex)+'*')
                xcom=True
            else: #Wenn x schon bestätigt, dann Erstellen des neuen Feldes mit eingegebener Größe
                xcom=False
                global fields
                global breiten
                global hohen
                global reali
                fields=[np.zeros([newsizey, newsizex])]
                hohen=[newsizey]
                breiten=[newsizex]
                reali.set_val(0)
                num8a.set_title('xsize=')
                num9a.set_title('ysize=')
                newsizex=0
                newsizey=0
        num0b.on_clicked(num0func)
        num1b.on_clicked(num1func)
        num2b.on_clicked(num2func)
        num3b.on_clicked(num3func)
        num4b.on_clicked(num4func)
        num5b.on_clicked(num5func)
        num6b.on_clicked(num6func)
        num7b.on_clicked(num7func)
        num8b.on_clicked(num8func)
        num9b.on_clicked(num9func)
        delb.on_clicked(delnum)
        comb.on_clicked(comnum)
    NewF.on_clicked(NewFfunc)
        
    def pausefunc(val):
        """
        Die Funktion pausefunc setzt pause auf True, sodass die Animation angehalten wird
        args:
            val : Benötigt zum Ausführen mit Buttons
        kein return Wert
        """
        global pause
        pause=True
    
    def startfunc(val):
        """
        Die Funktion startfunc setzt pause und editing auf False, sodass die Animation fortgeführt wird.
        Die Bearbeitung des Feldes ist nun nicht mehr möglich.
        args:
            val : Benötigt zum Ausführen mit Buttons
        kein return Wert
        """
        global pause
        pause=False
        global editing
        editing=False
    
    def one_stepfunc(val):
        """
        Die Funktion one_stepfunc setzt single_step auf True, sodass die Animation einen Schritt lang ausgeführt wird.
        args:
            val : Benötigt zum Ausführen mit Buttons
        kein return Wert
        """
        global single_step
        single_step=True
        global editing
        editing=False
    
    def one_b_stepfunc(val):
        """
        Die Funktion one_b_stepfunc setzt single_b_step auf True, sodass die Animation einen Schritt zurück geht.
        args:
            val : Benötigt zum Ausführen mit Buttons
        kein return Wert
        """
        global single_b_step
        single_b_step=True
        global editing
        editing=False
    
    #Options-Buttons einrichten
    GenOpt.on_clicked(genopt)
    ColOpt.on_clicked(colors)
    FieldOpt.on_clicked(edfi)
    
    pauseb.on_clicked(pausefunc)
    startb.on_clicked(startfunc)
    singlestepb.on_clicked(one_stepfunc)
    singlebstepb.on_clicked(one_b_stepfunc)
    
    pause=False
    editing=False
    single_step=False
    single_b_step=False
    
    def animate(i):
        '''
        Die Funktion animate wird verwendet, um das Feld animiert auszugeben. Sie gibt den Plot (genauer die Imschow) in abhängigkeit des Schrittes mit dem entsprechenden Feld aus.
        args:
            i : Nummer des Schrittes, allerdings wird eine interne Zählung verwendet. Die Eingabe ist somit irrelevant, gemäß den Einstellungen wird die Schrittnummer mit jedem Aufruf der Funktion erhöht. 
        returns line
        '''
        global pause
        global editing
        global single_step
        global single_b_step
        global bouncon
        global reali
        #Zählung der Schrittnummer in Abhängigkeit von pause, editing, single_step, single_b_step
        if pause==False and editing==False:
            if reali.val<interval:
                reali.set_val(int(reali.val)+1)
            else:
                reali.set_val(0)
        if single_step:
            if reali.val<interval:
                reali.set_val(int(reali.val)+1)
            else:
                reali.set_val(0)
            single_step=False
        if single_b_step:
            if reali.val!=0:
                reali.set_val(int(reali.val)-1)
            else:
                reali.set_val(interval)
            single_b_step=False
        if reali.val>=len(fields): #Falls der Wert manuell zu hoch gesetzt worden sein sollte, wird der höchst mögliche, bereits berechnete Wert gewählt.
            reali.set_val(len(fields))
        i=int(reali.val)
        if len(fields)>=i+1: #Berechnete Werte
            newfield=fields[i]
            hohe=hohen[i]
            breite=breiten[i]
        else: #Neue Berechnung der Daten
            field=fields[i-1]         
            newfield, breite, hohe = step(field, livecrit, borncrit, bouncon)
            fields.append(np.array(newfield)) #Speicherung für spätere Durchläufe
            hohen.append(int(hohe))
            breiten.append(int(breite))
        ax.set_xlim([-0.5, breite-0.5]) #Anpassen der Ansicht
        ax.set_ylim([-0.5, hohe-0.5])
        ax.clear()
        line = ax.imshow(newfield, interpolation='nearest') #Erstellen der neuen imshow
        ax.set_title("$step$ = {}".format(i))
        return line
    
    ani = animation.FuncAnimation(fig, animate, np.arange(0, interval+1), interval=speed, repeat=True)
    
    show()
    

In [9]:
GameOfLife(steps=100, speed=1)

##Funktion zum Import der Benutzereingabe

In [7]:
def GoLdataimport():
    """
    Die Funktion GoLdataimport importiert die Bedingungen aus den mit prepGoL erstellten Textdateien. Die Form ist wie bei prepGoL beschrieben zu halten.
    Sie benötigt keine Argumente.
    returns (np.arrays)
        size : Größe des Feldes
        bouncon : Randbedingung
        incon : Anfangsbedingung
        rule : Regel (erste Zeile: bornrule, zweite Zeile: liverule)
    """
    size = np.loadtxt("GameOfLifeSize.dat", dtype=int)
    bouncon = np.loadtxt("GameOfLifeBouncon.dat", dtype=int)
    incon = np.loadtxt("GameOfLifeIncon.dat", dtype=int)
    rule = np.loadtxt("GameOfLifeRule.dat", dtype=int)
    return size, bouncon, incon, rule

##Funktion, welche den Datenimport und den Plot vornimmt

In [8]:
def GameOfLife(steps=10, speed=1):
    """
    Die Funktion GameOfLife erzeugt eine graphisch animierte Ausgabe der Entwicklung der Anfangsmatrix nach dem Game of Life.
    Die Bedingungen sind durch Textdateien vorzugeben (siehe auch prepGoL). 
    Sollte dies nicht geschehen, werden die Standartwerte der Funktion prepGoL verwendet.
    kwargs:
        steps=10 : Anzahl der durchzufürenden Schritte
        speed=1 : Geschwindigkeit, mit der die Aimation erfolgen soll
                    100*10**(1-speed) ist die Zeit in ms pro Schritt (empfohlen: Wert zwischen 0 (langsam) und 1 (schnell))
    """
    try:
        size, bouncon, incon, rule = GoLdataimport()
        liverule = [rule[1][i] for i in range(len(rule[1])) if rule[1][i]!=0 or i==0]
        bornrule = [rule[0][i] for i in range(len(rule[0])) if rule[0][i]!=0 or i==0]
    except: #ggf. Setzen von Standart-Werten
        size = [9,9]
        bouncon = 1
        incon = np.array([[3,3],[3,4],[3,5]])
        liverule = [3]
        bornrule=[2,3]    
    
    field =  np.zeros(size)
    for i in range(incon.shape[0]):
        field[incon[i][1], incon[i][0]]=1
    
    speed=100*10**(1-speed)
    
    GoLplot(field, GoLStep, liverule, bornrule, bouncon, steps, speed)
    return